# Task 1: Import Libraries

# Task 2: Load the Dataset

# Task 3: Visualize the Dataset

# Task 4: Create a Mapping of Class Names to Index

# Task 5: Load the Preprocessor for the Dataset

# Task 6: Define Data Augmentations

# Task 7:  Implement Data Transformation

# Task 8: Collate the Function for DataLoader

# Task 9: Create a Model

# Task 10: Define a Metric for the Model

# Task 11: Set Up Trainer Arguments

# Task 12: Create a Trainer Object

# Task 13: Evaluate the Model Before Training

# Task 14: Train the Model

# Task 15: Visualize the Performance in TensorBoard

# Task 16: Evaluate the Model

# Task 17: Set Up the Confusion Matrix

# Task 18: Save the Model and Metrics

# Task 19: Set Up an Inference for the Model